In [159]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [160]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [161]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [162]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

In [163]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [164]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [165]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [166]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [167]:
compras = (pd.concat
                ([
                pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'),
                productos_sin_compra
               ])
        )


cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras[cols_to_Int64] = compras[cols_to_Int64].astype('Int64')

compras['tolerance_order_date'] = compras['order_date'].dt.normalize() - pd.Timedelta(days=5)

In [168]:
costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        # tolerance = pd.Timedelta(days=1),
        
        direction = 'backward')
)

# Checks

In [169]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')


Hay proveedores no calificados


,order_id,order_name,order_state,order_date,partner_id,partner_name,partner_fact_ref,partner_fact_date,capturista,vendedora
2,3150,P03136,purchase,2024-06-28 00:04:29,5272,Feyco,C 26910,2024-06-27,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez


In [170]:
check2 = costo_venta[costo_venta['product_price'].isna()][['product_id', 'product_name_x']]
print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 22 renglones sin costo de la venta.
Los productos sin costo son los siguientes:


,product_id,product_name_x
10061,14757,Nuevo *0*
10282,28586,Anticipo
13416,28638,Servicios de Facturación
18005,4,Anticipo (PdV)
63264,29607,"Bis. Dex #7225 4x4"" Arq C/Bal NS (Pz) *5435*"


# Pruebas

In [201]:
prod_varios_prov = (compras[['partner_id', 'partner_name', 'product_id_pp', 'product_name']]
                    .sort_values('partner_id').groupby('product_id_pp')
                    .agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
                    )

prod_varios_prov.columns = ['product_name', 'mean', 'first']
prod_varios_prov['diff'] = prod_varios_prov['mean']  == prod_varios_prov['first']

prod_varios_prov_ids = prod_varios_prov.loc[prod_varios_prov['diff'] == False].reset_index()['product_id_pp']

ventas_prod_varios_prov = ventas_año[ventas_año['product_id'].isin(prod_varios_prov_ids)]
compras_prod_varios_prov = compras[compras['product_id_pp'].isin(prod_varios_prov_ids)]

ventas_año_diff = ventas_año[~ventas_año['product_id'].isin(prod_varios_prov_ids)]

In [207]:
# Checar si es mejor primero (ronda 1 de concatenación), sacar de las compras los 212 'prod_varios_prov_ids', para que se quede el resto del DF en esa columna como NaN y en una
# segunda vuelta calificar por niveles el resto de los renglones NaN

v = pd.concat([ventas_año_diff, ventas_prod_varios_prov]).sort_values(by='invoice_date')

### 2da vuelta al costo de ventas

In [215]:
compras_prod_varios_prov.loc[:, 'product_qty'] = compras_prod_varios_prov['product_qty'].astype('Float64')
ventas_prod_varios_prov.loc[:, 'salesperson_name'] = ventas_prod_varios_prov['salesperson_name'].astype('object')

compras_costo_venta_prod_varios_prov = (
    pd.merge_asof(
        left = compras_prod_varios_prov.sort_values('order_date'), 
        right = ventas_prod_varios_prov.sort_values('invoice_date'),
        
        left_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        right_by = ['product_id', 'salesperson_name', 'quantity'], 
        
        left_on = 'order_date', 
        right_on = 'invoice_date', 

        tolerance = pd.Timedelta(days=5),
        
        direction = 'nearest')
)

In [222]:
len(ventas_prod_varios_prov)-764

3400

In [241]:
len(compras_costo_venta_prod_varios_prov)-764

664

In [240]:
compras_costo_venta_prod_varios_prov[~compras_costo_venta_prod_varios_prov['fact_doc_id'].isna()]

,line_id,order_id,order_name,order_date,partner_id_x,partner_name_x,product_id_pp,product_name_x,product_qty,product_price,...,warehouse,fact_line_id,product_id,product_name_y,prod_codigo,prod_linea,quantity,price_unit,discount,price_subtotal
47,2,1,P00001 (SJCFCO 68),2024-01-03 00:05:53,5326,Madereria El Pino De Los Cabos,14272,[13508] Tablon Americano 2x12x12 *13508*,2.00,910.34,...,A2,2927,14272,Tablon Americano 2x12x12 *13508*,13508,00MAD,2.00,"1,551.74",0.00,"3,103.48"
48,3,1,P00001 (SJCFCO 68),2024-01-03 00:05:53,5326,Madereria El Pino De Los Cabos,14147,[3637] Triplay Pino 3/4 4'*8' 1/Cara *3637*,20.00,646.55,...,A2,2576,14147,Triplay Pino 3/4 4'*8' 1/Cara *3637*,3637,00MAD,20.00,969.82,0.00,"19,396.40"
49,5,2,P00002 (SJCFCO 23),2024-01-03 00:09:24,5326,Madereria El Pino De Los Cabos,13371,[12749] Triplay Pino 1/4 4'*8' 1/Cara *12749*,10.00,267.25,...,A2,11002,13371,Triplay Pino 1/4 4'*8' 1/Cara *12749*,12749,00MAD,10.00,390.00,0.00,"3,900.00"
52,63,48,P00041 (MPIFCO 125),2024-01-04 00:03:53,5296,Madereria Pichonchas,14772,[16313] Pegamento Titebond Verde1L *16313*,6.00,245.69,...,A1,5709,14772,Pegamento Titebond Verde1L *16313*,16313,000CE,6.00,413.79,0.00,"2,482.74"
53,62,48,P00041 (MPIFCO 125),2024-01-04 00:03:53,5296,Madereria Pichonchas,14147,[3637] Triplay Pino 3/4 4'*8' 1/Cara *3637*,2.00,646.55,...,A1,5706,14147,Triplay Pino 3/4 4'*8' 1/Cara *3637*,3637,00MAD,2.00,969.82,0.00,"1,939.64"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420,19094,3131,P03117 (SJCFCO 16621),2024-06-26 23:42:17,5326,Madereria El Pino De Los Cabos,13331,[16320] Triplay Birch 1/4 4'*8' *16320*,2.00,267.24,...,A2,413989,13331,Triplay Birch 1/4 4'*8' *16320*,16320,00MAD,2.00,413.81,0.00,827.62
1421,19093,3131,P03117 (SJCFCO 16621),2024-06-26 23:42:17,5326,Madereria El Pino De Los Cabos,14075,[40032] Triplay Birch 5/8 4'*8' *40032*,3.00,616.38,...,A2,413988,14075,Triplay Birch 5/8 4'*8' *40032*,40032,00MAD,3.00,955.39,0.00,"2,866.17"
1424,19110,3141,P03127 (37206),2024-06-27 18:44:39,16544,Framing and Finishing,28388,[1437] Base Coat Cemenquin 20 kg *1437*,5.00,205.00,...,A1,414614,28388,Base Coat Cemenquin 20 kg *1437*,1437,000CE,5.00,805.00,0.00,"4,025.00"
1425,19113,3142,P03128 (MPIFCO 13013),2024-06-27 18:46:11,5296,Madereria Pichonchas,13781,"[14145] Tabla Americana 1"" X 12"" X 12' *14145*",4.00,403.45,...,A1,409125,13781,"Tabla Americana 1"" X 12"" X 12' *14145*",14145,00MAD,4.00,620.69,0.00,"2,482.76"


Después de corregir el costo de la venta... las siguientes líneas ya son de utilidades

In [ ]:
# #Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

# for i in range(len(ventas_año['price_subtotal'])):
#     ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [ ]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]